In [1]:
# %pip install sec-api
# %pip install pdfkit
# %pip install wkhtmltopdf

In [47]:
from sec_api import ExtractorApi
import pdfkit
import pandas as pd
import os
from sec_api import RenderApi
from pathlib import Path
import multiprocessing

In [7]:
os. getcwd()

'/Users/satoshiido/Documents/programming/RA/10k_filling'

In [21]:
# load Russell 3000 holdings CSV into a dataframe
holdings = pd.read_csv('./mikecik_2015/mikecik_2015.csv')
display(holdings.head())
display(holdings.tail())

,cik,Ticker
0,1090872,A
1,815094,ABMD
2,1427925,ACRX
3,926282,ADTN
4,1506928,AVGR


,cik,Ticker
45,1429560,TRVN
46,1519061,TSE
47,1105705,TWX
48,1124610,VMW
49,105770,WST


In [48]:
from sec_api import QueryApi
api_key = "34686d55d1115b6b415141f1501c0d8c66ce5ef2cc31d77b797dbc028f14e22b"
queryApi = QueryApi(api_key=api_key)

In [23]:
# create batches of tickers: [[A,B,C], [D,E,F], ...]
# a single batch has a maximum of max_length_of_batch tickers
def create_batches(tickers = [], max_length_of_batch = 100):
  batches = [[]]

  for ticker in tickers:
    if len(batches[len(batches)-1]) == max_length_of_batch:
      batches.append([])

    batches[len(batches)-1].append(ticker)

  return batches


batches = create_batches(list(holdings['Ticker']))


In [49]:
def download_10K_metadata(tickers = [], start_year = 2015, end_year = 2015):
  if Path('./mikecik_2015/metadata.csv').is_file():
    result = None
    result = pd.read_csv('./mikecik_2015/metadata.csv')
    return result

  print('Starting download process')

  # create ticker batches, with 100 tickers per batch
  batches = create_batches(tickers)
  frames = []

  for year in range(start_year, end_year + 1):
    for batch in batches:
      tickers_joined = ', '.join(batch)
      ticker_query = 'ticker:({})'.format(tickers_joined)

      query_string = '{ticker_query} AND filedAt:[{start_year}-01-01 TO {end_year}-12-31] AND formType:"10-K" AND NOT formType:"10-K/A" AND NOT formType:NT'.format(ticker_query=ticker_query, start_year=year, end_year=year)

      query = {
        "query": { "query_string": { 
            "query": query_string,
            "time_zone": "America/New_York"
        } },
        "from": "0",
        "size": "200",
        "sort": [{ "filedAt": { "order": "desc" } }]
      }

      response = queryApi.get_filings(query)

      filings = response['filings']

      metadata = list(map(lambda f: {'ticker': f['ticker'], 
                                     'cik': f['cik'], 
                                     'formType': f['formType'], 
                                     'filedAt': f['filedAt'], 
                                     'filingUrl': f['linkToFilingDetails']}, filings))

      df = pd.DataFrame.from_records(metadata)

      frames.append(df)

    print('Downloaded metadata for year', year)


  result = pd.concat(frames)
  result.to_csv('metadata.csv', index=False)

  number_metadata_downloaded = len(result)
  print('Downloaded completed. Metadata downloaded for {} filings.'.format(number_metadata_downloaded))

  return result

In [50]:
tickers = list(holdings['Ticker'])

metadata = download_10K_metadata(tickers=tickers, start_year=2015, end_year=2015)
metadata.head()

In [59]:
metadata.head()

,ticker,cik,formType,filedAt,filingUrl
0,A,1090872,10-K,2015-12-18T19:33:31-05:00,https://www.sec.gov/Archives/edgar/data/109087...
1,NDSN,72331,10-K,2015-12-15T11:36:58-05:00,https://www.sec.gov/Archives/edgar/data/72331/...
2,RSTI,1019361,10-K,2015-11-30T16:41:39-05:00,https://www.sec.gov/Archives/edgar/data/101936...
3,BDX,10795,10-K,2015-11-25T11:35:36-05:00,https://www.sec.gov/Archives/edgar/data/10795/...
4,FFIV,1048695,10-K,2015-11-06T13:28:33-05:00,https://www.sec.gov/Archives/edgar/data/104869...


In [57]:
len(metadata)

42

In [ ]:
# this part below is where we want to change using extractorApi
import requests
import datetime

# Filing Render & Download API:
# api key
## https://sec-api.io/login
## https://github.com/janlukasschroeder/sec-api-python/blob/master/README.md#filing-render--download-api
## https://pypi.org/project/sec-api/#filing-render--download-api
extractorApi = ExtractorApi("34686d55d1115b6b415141f1501c0d8c66ce5ef2cc31d77b797dbc028f14e22b")


# get the original HTML of section 1 "Business"
section1_html = extractorApi.get_section(filing_url_10k, "1", "html")
# get the original HTML of section 1A "Risk Factors"
section1a_html = extractorApi.get_section(filing_url_10k, "1A", "html")


# this part below is also the part where we want to change (we want to convert html to pdf file)
for i in range(0, len(metadata)):
    response = requests.get(api_url.iloc[i])
    # save the PDF to a local file "filing.pdf"

    # add year to the name
    with open(file_name_cik.iloc[i] + file_name.iloc[i]+".pdf", 'wb') as f:
        f.write(response.content)

In [25]:
API_ENDPOINT = "https://api.sec-api.io/filing-reader"
API_KEY = "458301555a0838acf5e8eca62a944cce9186d6612e35a3fb5e8fcfefef7fbf5cff"
filing_url = metadata['filingUrl'].replace('ix?doc=/', '')
api_url = API_ENDPOINT + "?token=" + API_KEY + "&url=" + filing_url + "&type=pdf"

NameError: name 'metadata' is not defined

In [53]:
# this part below is where we want to change using extractorApi
import requests
import datetime
API_ENDPOINT = "https://api.sec-api.io/filing-reader"
API_KEY = "34686d55d1115b6b415141f1501c0d8c66ce5ef2cc31d77b797dbc028f14e22b"
filing_url = metadata['filingUrl'].replace('ix?doc=/', '')
api_url = API_ENDPOINT + "?token=" + API_KEY + "&url=" + filing_url + "&type=pdf"
file_name = filing_url.str.rsplit("/", n=1, expand=True)[1]
file_name_cik = metadata['cik']

# this part below is also the part where we want to change (we want to convert html to pdf file)
for i in range(0, len(metadata)):
    response = requests.get(api_url.iloc[i])
    # save the PDF to a local file "filing.pdf"

    # add year to the name
    with open(file_name_cik.iloc[i] + file_name.iloc[i]+".pdf", 'wb') as f:
        f.write(response.content)

In [3]:
# 10-K example

# Tesla 10-K filing
filing_url_10k = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm"

# get the standardized and cleaned text of section 1 "Business"
section1_text = extractorApi.get_section(filing_url_10k, "1", "text")
# get the standardized and cleaned text of section 1A "Risk Factors"
section1a_text = extractorApi.get_section(filing_url_10k, "1A", "text")



# get the original HTML of section 1 "Business"
section1_html = extractorApi.get_section(filing_url_10k, "1", "html")
# get the original HTML of section 1A "Risk Factors"
section1a_html = extractorApi.get_section(filing_url_10k, "1A", "html")

In [6]:
# Configure pdfkit with path to wkhtmltopdf if it's not in PATH
# pdfkit_config = pdfkit.configuration(wkhtmltopdf="/path/to/wkhtmltopdf")

# Combine the HTML content into one large HTML string
combined_html = section1_html + section1a_html

# Output filename for the PDF
output_pdf = 'section1_1a.pdf'

# Convert HTML to PDF
# pdfkit.from_string(combined_html, output_pdf)
pdfkit.from_string(combined_html, output_pdf)

# print(f"The PDF file has been created: {output_pdf}")

True

In [ ]:

# api_key='cfa0929a6e4b23d8a7f882039072195652578fdcf7cc0ee9732ad712ff9834f1'

queryApi = QueryApi(api_key=api_key)

def download_10K_metadata(tickers = [], start_year = 2015, end_year = 2015):
  if Path('metadata.csv').is_file():
    result = pd.read_csv('metadata.csv')
    return result

  print('Starting download process')

  # create ticker batches, with 100 tickers per batch
  batches = create_batches(tickers)
  frames = []

  for year in range(start_year, end_year + 1):
    for batch in batches:
      tickers_joined = ', '.join(batch)
      ticker_query = 'ticker:({})'.format(tickers_joined)

      query_string = '{ticker_query} AND filedAt:[{start_year}-01-01 TO {end_year}-12-31] AND formType:"10-K" AND NOT formType:"10-K/A" AND NOT formType:NT'.format(ticker_query=ticker_query, start_year=year, end_year=year)

      query = {
        "query": { "query_string": { 
            "query": query_string,
            "time_zone": "America/New_York"
        } },
        "from": "0",
        "size": "200",
        "sort": [{ "filedAt": { "order": "desc" } }]
      }

      response = queryApi.get_filings(query)

      filings = response['filings']

      metadata = list(map(lambda f: {'ticker': f['ticker'], 
                                     'cik': f['cik'], 
                                     'formType': f['formType'], 
                                     'filedAt': f['filedAt'], 
                                     'filingUrl': f['linkToFilingDetails']}, filings))

      df = pd.DataFrame.from_records(metadata)

      frames.append(df)

    print('Downloaded metadata for year', year)


  result = pd.concat(frames)
  result.to_csv('metadata.csv', index=False)

  number_metadata_downloaded = len(result)
  print('Downloaded completed. Metadata downloaded for {} filings.'.format(number_metadata_downloaded))

  return result


tickers = list(holdings['Ticker'])

metadata = download_10K_metadata(tickers=tickers, start_year=2015, end_year=2015)

# this part below is where we want to change using extractorApi
import requests
import datetime
API_ENDPOINT = "https://api.sec-api.io/filing-reader"
API_KEY = "458301555a0838acf5e8eca62a944cce9186d6612e35a3fb5e8fcfefef7fbf5cff"
filing_url = metadata['filingUrl'].replace('ix?doc=/', '')
api_url = API_ENDPOINT + "?token=" + API_KEY + "&url=" + filing_url + "&type=pdf"
file_name = filing_url.str.rsplit("/", n=1, expand=True)[1]
file_name_cik = metadata['cik']


# this part below is also the part where we want to change (we want to convert html to pdf file)
for i in range(0, len(metadata)):
    response = requests.get(api_url.iloc[i])
    # save the PDF to a local file "filing.pdf"

    # add year to the name
    with open(file_name_cik.iloc[i] + file_name.iloc[i]+".pdf", 'wb') as f:
        f.write(response.content)